# AutoML: Train "the best" Image Classification Multi-Label model for a 'Fridge items' dataset.

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](../../../resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](../../../resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create an `AutoML Image Classification Multilabel Training Job` with the 'image_classification_multilabel()' factory-function.
- Train the model using AmlCompute by submitting/running the AutoML training job
- Obtaing the model and score predictions with it

**Motivations** - This notebook explains how to setup and run an AutoML image classification-multilabel job. This is one of the nine ML-tasks supported by AutoML. Other ML-tasks are 'forecasting', 'classification', 'image object detection', 'nlp text classification', etc.

In this notebook, we go over how you can use AutoML for training an Image Classification Multi-Label model. We will use a small dataset to train the model, demonstrate how you can tune hyperparameters of the model to optimize model performance and deploy the model to use in inference scenarios. 

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [64]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential,AzureCliCredential
from azure.ai.ml import MLClient

from azure.ai.ml.automl import SearchSpace, ClassificationPrimaryMetrics
from azure.ai.ml.sweep import (
    Choice,
    Uniform,
    BanditPolicy,
)

from azure.ai.ml import automl

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a han
dle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [8]:

'''
try:
    raise Exception
    # Will check all different authentication options on system and try and use on of those
    credential = DefaultAzureCredential(verbose=True)
    # Check if given credential can get token successfully.
    #print(credential.get_token())
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()
    pass

'''


In [28]:
credential = AzureCliCredential()

In [29]:

ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    subscription_id = "3bef7a06-fecc-47fc-ba25-a7899d62e16c"
    resource_group = "Mini-Tumors-Germany"
    workspace = "Mini-Tumors-Workspace"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)


We could not find config.json in: . or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories.


# 2. MLTable with input Training Data

In order to generate models for computer vision tasks with automated machine learning, you need to bring labeled image data as input for model training in the form of an MLTable. You can create an MLTable from labeled training data in JSONL format. If your labeled training data is in a different format (like, pascal VOC or COCO), you can use a conversion script to first convert it to JSONL, and then create an MLTable. Alternatively, you can use Azure Machine Learning's [data labeling tool](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-image-labeling-projects) to manually label images, and export the labeled data to use for training your AutoML model.

In this notebook, we use a toy dataset called Fridge Objects, which consists of 128 images of 4 labels of beverage container {can, carton, milk bottle, water bottle} photos taken on different backgrounds. It also includes a labels file in .csv format. This is one of the most common data formats for Image Classification Multi-Label: one csv file that contains the mapping of labels to a folder of images.

All images in this notebook are hosted in [this repository](https://github.com/microsoft/computervision-recipes) and are made available under the [MIT license](https://github.com/microsoft/computervision-recipes/blob/master/LICENSE).

**NOTE:** In this PRIVATE PREVIEW we're defining the MLTable in a separate folder and .YAML file.
In later versions, you'll be able to do it all in Python APIs.

## 2.2. Upload the images to Datastore through an AML Data asset (URI Folder)

In order to use the data for training in Azure ML, we upload it to our default Azure Blob Storage of our  Azure ML Workspace.

[Check this notebook for AML data asset example](https://github.com/Azure/azureml-examples/blob/de54247989beee62d1fd89c2b6f1b89f49ef2f47/sdk/python/assets/data/data.ipynb)



In [30]:
# Uploading image files by creating a 'data asset URI FOLDER':

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

my_data = Data(
    path="../data/converted",
    type=AssetTypes.URI_FOLDER,
    description="Images 128x128 Scaled Outliers Removed",
    name="mini-tumors-images-scaled",
)

uri_folder_data_asset = ml_client.data.create_or_update(my_data)

print(uri_folder_data_asset)
print("")
print("Path to folder in Blob Storage:")
print(uri_folder_data_asset.path)

Uploading converted (75.8 MBs): 100%|█| 75804395/75804395 [00:27<00:00, 2780869.




Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'mini-tumors-images-scaled', 'description': 'Images 128x128 Scaled Outliers Removed', 'tags': {}, 'properties': {}, 'id': '/subscriptions/3bef7a06-fecc-47fc-ba25-a7899d62e16c/resourceGroups/Mini-Tumors-Germany/providers/Microsoft.MachineLearningServices/workspaces/Mini-Tumors-Workspace/data/mini-tumors-images-scaled/versions/5', 'Resource__source_path': None, 'base_path': '/Users/louis.heath/Documents/School/mini-tumors/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x10cb92d00>, 'serialize': <msrest.serialization.Serializer object at 0x10979d9d0>, 'version': '5', 'latest_version': None, 'path': 'azureml://subscriptions/3bef7a06-fecc-47fc-ba25-a7899d62e16c/resourcegroups/Mini-Tumors-Germany/workspaces/Mini-Tumors-Workspace/datastores/workspaceblobstore/paths/LocalUpload/73f9826

## 2.3. Convert the downloaded data to JSONL

In this example, the fridge object dataset is annotated in the CSV file, where each image corresponds to a line. It defines a mapping of the filename to the labels. Since this is a multi-label classification problem, each image can be associated to multiple labels. In order to use this data to create an AzureML MLTable, we first need to convert it to the required JSONL format. Please refer to the [documentation on how to prepare datasets](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-prepare-datasets-for-automl-images).


The following script is creating two .jsonl files (one for training and one for validation) in the corresponding MLTable folder. The train / validation ratio corresponds to 20% of the data going into the validation file.

In [32]:
# Create labels.csv in correct format
#filename,labels
#1.jpg,carton
#2.jpg,milk_bottle
import numpy as np
import pandas as pd

labels = np.load('../data/converted/images/labels.npy')

new_labs = pd.DataFrame(columns=['filename','labels'])


new_labs['filename'] = [ f'img{i}.png' for i in range(len(labels))]
new_labs['labels'] = labels

new_labs = new_labs.set_index('filename')

new_labs.to_csv('../data/converted/images/labels.csv')
new_labs.to_csv('../data/converted/labels.csv')
#for i in len(labels):
#    

### Remember to move to /images subforlder!

In [47]:
import json
import os

# In the big data folder
src_images = "../data/converted"

# We'll copy each JSONL file within its related MLTable folder
# Do this locally in ./data for notebooks
training_mltable_path = "../data/training-mltable-folder/"
validation_mltable_path = "../data/validation-mltable-folder/"

# Reasonable
train_validation_ratio = 5

# Path to the training and validation files
train_annotations_file = os.path.join(training_mltable_path, "train_annotations.jsonl")
validation_annotations_file = os.path.join(
    validation_mltable_path, "validation_annotations.jsonl"
)

'''
Lets not reinvent the wheel for now
    "image_details":
      {
          "format": "png",
          "width": "128px",
          "height": "1286px"
      },

'''

# Baseline of json line dictionary
json_line_sample = {
    "image_url": uri_folder_data_asset.path,
    "label": [],
}

print(json_line_sample["image_url"])


labelFile = os.path.join(src_images, "labels.csv")

# Read each annotation and convert it to jsonl line
with open(train_annotations_file, "w") as train_f:
    with open(validation_annotations_file, "w") as validation_f:
        with open(labelFile, "r") as labels:
            for i, line in enumerate(labels):
                # Skipping the title line and any empty lines.
                if i == 0 or len(line.strip()) == 0:
                    continue
                line_split = line.strip().split(",")
                if len(line_split) != 2:
                    print("Skipping the invalid line: {}".format(line))
                    continue
                json_line = dict(json_line_sample)
                json_line["image_url"] += f"images/{line_split[0]}"
                json_line["label"] = line_split[1].strip().split(" ")[0]

                # Nothe the train validation split is not random
                if i % train_validation_ratio == 0:
                    # validation annotation
                    validation_f.write(json.dumps(json_line) + "\n")
                else:
                    # train annotation
                    train_f.write(json.dumps(json_line) + "\n")

azureml://subscriptions/3bef7a06-fecc-47fc-ba25-a7899d62e16c/resourcegroups/Mini-Tumors-Germany/workspaces/Mini-Tumors-Workspace/datastores/workspaceblobstore/paths/LocalUpload/73f9826a213885fe962e833cc0ee47f1/converted/


## 2.4. Create MLTable data input

Create MLTable data input using the jsonl files created above.

In [48]:

# Training MLTable defined locally, with local data to be uploaded
my_training_data_input = Input(type=AssetTypes.MLTABLE, path=training_mltable_path)

# Validation MLTable defined locally, with local data to be uploaded
my_validation_data_input = Input(type=AssetTypes.MLTABLE, path=validation_mltable_path)


'''
# WITH REMOTE PATH: If available already in the cloud/workspace-blob-store
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/vision-classification/train")
my_validation_data_input = Input(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/vision-classification/valid")
'''

'\n# WITH REMOTE PATH: If available already in the cloud/workspace-blob-store\nmy_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/vision-classification/train")\nmy_validation_data_input = Input(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/vision-classification/valid")\n'

To create data input from TabularDataset created using V1 sdk, specify the `type` as `AssetTypes.MLTABLE`, `mode` as `InputOutputModes.DIRECT` and `path` in the following format `azureml:<tabulardataset_name>:<version>`.

In [36]:
'''
# Training MLTable withv1 TabularDataset
my_training_data_input = Input(
    type=AssetTypes.MLTABLE, path="azureml:multilabelFridgeObjectsTrainingDataset:1",
    mode=InputOutputModes.DIRECT
)

# Validation MLTable with v1 TabularDataset
my_validation_data_input = Input(
    type=AssetTypes.MLTABLE, path="azureml:multilabelFridgeObjectsValidationDataset:1",
    mode=InputOutputModes.DIRECT
)
'''

'\n# Training MLTable withv1 TabularDataset\nmy_training_data_input = Input(\n    type=AssetTypes.MLTABLE, path="azureml:multilabelFridgeObjectsTrainingDataset:1",\n    mode=InputOutputModes.DIRECT\n)\n\n# Validation MLTable with v1 TabularDataset\nmy_validation_data_input = Input(\n    type=AssetTypes.MLTABLE, path="azureml:multilabelFridgeObjectsValidationDataset:1",\n    mode=InputOutputModes.DIRECT\n)\n'

# 3. Compute target setup

You will need to provide a [Compute Target](https://docs.microsoft.com/en-us/azure/machine-learning/concept-azure-machine-learning-architecture#computes) that will be used for your AutoML model training. AutoML models for image tasks require [GPU SKUs](https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-gpu) such as the ones from the NC, NCv2, NCv3, ND, NDv2 and NCasT4 series. We recommend using the NCsv3-series (with v100 GPUs) for faster training. Using a compute target with a multi-GPU VM SKU will leverage the multiple GPUs to speed up training. Additionally, setting up a compute target with multiple nodes will allow for faster model training by leveraging parallelism, when tuning hyperparameters for your model.


Sizes :https://learn.microsoft.com/en-us/azure/machine-learning/concept-compute-target#supported-vm-series-and-sizes


Regional availability and pricing: https://azure.microsoft.com/en-us/pricing/details/machine-learning/

In [50]:
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

compute_name = "gpu-cluster"

try:
    _ = ml_client.compute.get(compute_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name=compute_name,
        type="amlcompute",
        size="Standard_NC6",
        idle_time_before_scale_down=120,
        min_instances=0,
        max_instances=1,
    )
    begin_create_or_update.begin_create_or_update(compute_config).result()

Found existing compute target.


## 4.1. Using default hyperparameter values for the specified algorithm
Before doing a large sweep to search for the optimal models and hyperparameters, we recommend trying the default values for a given model to get a first baseline. Next, you can explore multiple hyperparameters for the same model before sweeping over multiple models and their parameters. This allows an iterative approach, as with multiple models and multiple hyperparameters for each (as we showcase in the next section), the search space grows exponentially, and  you need more iterations to find optimal configurations.

Following functions are used to configure the AutoML image job:
### image_classification() function parameters:
The `image_classification()` factory function allows user to configure the training job.

- `compute` - The compute on which the AutoML job will run. In this example we are using a compute called 'gpu-cluster' present in the workspace. You can replace it any other compute in the workspace.
- `experiment_name` - The name of the experiment. An experiment is like a folder with multiple runs in Azure ML Workspace that should be related to the same logical machine learning experiment.
- `name` - The name of the Job/Run. This is an optional property. If not specified, a random name will be generated.
- `primary_metric` - The metric that AutoML will optimize for model selection.
- `target_column_name` - The name of the column to target for predictions. It must always be specified. This parameter is applicable to 'training_data' and 'validation_data'.
- `training_data` - The data to be used for training. It should contain both training feature columns and a target column. Optionally, this data can be split for segregating a validation or test dataset. 
You can use a registered MLTable in the workspace using the format '<mltable_name>:<version>' OR you can use a local file or folder as a MLTable. For e.g Input(mltable='my_mltable:1') OR Input(mltable=MLTable(local_path="./data"))
The parameter `training_data` must always be provided.


### set_limits() function parameters:
This is an optional configuration method to configure limits parameters such as timeouts.      
    
- `timeout_minutes` - Maximum amount of time in minutes that the whole AutoML job can take before the job terminates. If not specified, the default job's total timeout is 6 days (8,640 minutes).
- `max_trials` - Parameter for maximum number of configurations to sweep. Must be an integer between 1 and 1000. When exploring just the default hyperparameters for a given model algorithm, set this parameter to 1. Default value is 1.
- `max_concurrent_trials` - Maximum number of runs that can run concurrently. If not specified, all runs launch in parallel. If specified, must be an integer between 1 and 100.  Default value is 1.
    NOTE: The number of concurrent runs is gated on the resources available in the specified compute target. Ensure that the compute target has the available resources for the desired concurrency.
    
### set_training_parameters() function parameters:
This is an optional configuration method to configure fixed settings or parameters that don't change during the parameter space sweep. Some of the key parameters of this function are:

- `model_name` - The name of the ML algorithm that we want to use in training job. Please refer to this [documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-image-models?tabs=CLI-v2#supported-model-algorithms) for supported model algorithm.
- `number_of_epochs` - The number of training epochs. It must be positive integer (default value is 15).
- `layers_to_freeze` - The number of layers to freeze in model for transfer learning. It must be a positive integer (default value is 0).
- `early_stopping` - It enable early stopping logic during training, It must be boolean value (default is True).   
- `optimizer` - Type of optimizer to use in training. It must be either sgd, adam, adamw (default is sgd).
- `distributed` - It enable distributed training if compute target contain multiple GPUs. It must be boolean value (default is True).
    
If you wish to use the default hyperparameter values for a given algorithm (say `vitb16r224`), you can specify the job for your AutoML Image runs as follows:

In [51]:
# general job parameters
exp_name = "Mini-Tumor-Classification-AML_V1"

In [52]:
image_classification_job = automl.image_classification(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name="label",
)

image_classification_job.set_limits(timeout_minutes=60)

image_classification_job.set_training_parameters(model_name="seresnext")

### Submitting an AutoML job for Computer Vision tasks
Once you've configured your job, you can submit it as a job in the workspace in order to train a vision model using your training dataset.

In [53]:
#Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(image_classification_job)

print(f"Created job: {returned_job}")

Uploading training-mltable-folder (1.58 MBs): 100%|█| 1582376/1582376 [00:00<00:


Uploading validation-mltable-folder (0.4 MBs): 100%|█| 395587/395587 [00:00<00:0




Created job: ImageClassificationJob({'log_verbosity': <LogVerbosity.INFO: 'Info'>, 'target_column_name': 'label', 'validation_data_size': None, 'task_type': <TaskType.IMAGE_CLASSIFICATION: 'ImageClassification'>, 'training_data': {'type': 'mltable', 'path': 'azureml://datastores/workspaceblobstore/paths/LocalUpload/8510c5277a97e268ab28ae5e7675d1d3/training-mltable-folder'}, 'validation_data': {'type': 'mltable', 'path': 'azureml://datastores/workspaceblobstore/paths/LocalUpload/ef5ffaf8f7ab54efb70d3e073d3586f2/validation-mltable-folder'}, 'test_data': None, 'environment_id': None, 'environment_variables': None, 'outputs': {}, 'type': 'automl', 'status': 'NotStarted', 'log_files': None, 'name': 'boring_wheel_50pxv9bh28', 'description': None, 'tags': {}, 'properties': {'mlflow.source.git.repoURL': 'https://github.com/gakhromov/mini-tumors', 'mlflow.source.git.branch': 'azure_ml', 'mlflow.source.git.commit': '52033550558d310109afedfc4295b10eebf650fa', 'azureml.git.dirty': 'True'}, 'id': '

In [ ]:
ml_client.jobs.stream(returned_job.name)

RunId: boring_wheel_50pxv9bh28
Web View: https://ml.azure.com/runs/boring_wheel_50pxv9bh28?wsid=/subscriptions/3bef7a06-fecc-47fc-ba25-a7899d62e16c/resourcegroups/Mini-Tumors-Germany/workspaces/Mini-Tumors-Workspace


## 4.2. Hyperparameter sweeping for your AutoML models for computer vision tasks

When using AutoML for Images, you can perform a hyperparameter sweep over a defined parameter space to find the optimal model. In this example, we sweep over the hyperparameters for `vitb16r224` and `seresnext` models, choosing from a range of values for learning_rate, gradient_accumulation_step, validation_resize_size, etc., to generate a model with the optimal 'iou'. If hyperparameter values are not specified, then default values are used for the specified algorithm.

set_sweep function is used to configure the sweep settings:
### set_sweep() parameters:

- `sampling_algorithm` - Sampling method to use for sweeping over the defined parameter space. Please refer to this [documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-image-models?tabs=SDK-v2#sampling-methods-for-the-sweep) for list of supported sampling methods.
- `early_termination` - Early termination policy to end poorly performing runs. If no termination policy is specified, all configurations are run to completion. Please refer to this [documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-image-models?tabs=SDK-v2#early-termination-policies) for supported early termination policies.

We use Random Sampling to pick samples from this parameter space and try a total of 10 iterations with these different samples, running 2 iterations at a time on our compute target. Please note that the more parameters the space has, the more iterations you need to find optimal models.

We leverage the Bandit early termination policy which will terminate poor performing configs (those that are not within 20% slack of the best performing config), thus significantly saving compute resources.

For more details on model and hyperparameter sweeping, please refer to the [documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters).

In [90]:
image_classification_job = automl.image_classification(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name="label",
    primary_metric=ClassificationPrimaryMetrics.ACCURACY,
)

# TODO: Perhaps try accuracy for primary metric?

In [91]:
# TODO: Perhaps try accuracy?
#ClassificationPrimaryMetrics.AUC_WEIGHTED

In [100]:
image_classification_job.set_limits(
    timeout_minutes=1440, #12 hours
    max_trials=20,
    max_concurrent_trials=1,
)

In [101]:
ss_transformers =   SearchSpace(
              model_name="vitb16r224",
              # According to: https://learn.microsoft.com/en-us/azure/machine-learning/reference-automl-images-hyperparameters
              layers_to_freeze=Choice([0, 5, 9]),
              # 0 for no weighted loss, 1 for weighted loss with sqrt.(class_weights), 2 for weighted loss with class_weights.
              weighted_loss=Choice([0,2]),
              learning_rate=Choice([0.0001, 0.01, 0.01]),
              number_of_epochs=30,
              optimizer="sgd",
        )

ss_cnn = SearchSpace(
          model_name=Choice(["seresnext", "resnet50", "resnet152"]),
          # According to: https://learn.microsoft.com/en-us/azure/machine-learning/reference-automl-images-hyperparameters
          layers_to_freeze=Choice([0, 2]),
          # 0 for no weighted loss, 1 for weighted loss with sqrt.(class_weights), 2 for weighted loss with class_weights.
          weighted_loss=Choice([0,2]),
          learning_rate=Choice([0.0001, 0.001, 0.01]),
          optimizer= Choice(["sgd","adamw"]),
          number_of_epochs=30
        )

In [102]:
image_classification_job.extend_search_space([ss_cnn, ss_transformers]) # ss_transformers

In [103]:
from azure.ai.ml.sweep import MedianStoppingPolicy

image_classification_job.set_sweep(
    sampling_algorithm="Random",
    early_termination= MedianStoppingPolicy(delay_evaluation = 15, evaluation_interval = 2)
)

In [104]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    image_classification_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

Created job: ImageClassificationJob({'log_verbosity': <LogVerbosity.INFO: 'Info'>, 'target_column_name': 'label', 'validation_data_size': None, 'task_type': <TaskType.IMAGE_CLASSIFICATION: 'ImageClassification'>, 'training_data': {'type': 'mltable', 'path': 'azureml://datastores/workspaceblobstore/paths/LocalUpload/8510c5277a97e268ab28ae5e7675d1d3/training-mltable-folder'}, 'validation_data': {'type': 'mltable', 'path': 'azureml://datastores/workspaceblobstore/paths/LocalUpload/ef5ffaf8f7ab54efb70d3e073d3586f2/validation-mltable-folder'}, 'test_data': None, 'environment_id': None, 'environment_variables': None, 'outputs': {}, 'type': 'automl', 'status': 'NotStarted', 'log_files': None, 'name': 'jovial_fork_74gkzcrl8c', 'description': None, 'tags': {}, 'properties': {'mlflow.source.git.repoURL': 'https://github.com/gakhromov/mini-tumors', 'mlflow.source.git.branch': 'azure_ml', 'mlflow.source.git.commit': '52033550558d310109afedfc4295b10eebf650fa', 'azureml.git.dirty': 'True'}, 'id': '/

In [105]:
ml_client.jobs.stream(returned_job.name)

RunId: jovial_fork_74gkzcrl8c
Web View: https://ml.azure.com/runs/jovial_fork_74gkzcrl8c?wsid=/subscriptions/3bef7a06-fecc-47fc-ba25-a7899d62e16c/resourcegroups/Mini-Tumors-Germany/workspaces/Mini-Tumors-Workspace


Retrying due to transient client side error HTTPSConnectionPool(host='westus2-0.in.applicationinsights.azure.com', port=443): Max retries exceeded with url: /v2.1/track (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x143062d60>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')).
Retrying due to transient client side error HTTPSConnectionPool(host='westus2-0.in.applicationinsights.azure.com', port=443): Max retries exceeded with url: /v2.1/track (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x14307b820>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')).
Retrying due to transient client side error HTTPSConnectionPool(host='westus2-0.in.applicationinsights.azure.com', port=443): Max retries exceeded with url: /v2.1/track (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x14307b6a0>: Failed to establish a new 

ClientAuthenticationError: ERROR: The command failed with an unexpected error. Here is the traceback:
ERROR: HTTPSConnectionPool(host='login.microsoftonline.com', port=443): Max retries exceeded with url: /16bf80d2-3fa8-456d-8ca3-59cab29a2b99/oauth2/v2.0/token (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x107270fd0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/azure-cli/2.42.0/libexec/lib/python3.10/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/opt/homebrew/Cellar/azure-cli/2.42.0/libexec/lib/python3.10/site-packages/urllib3/util/connection.py", line 73, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socket.py", line 955, in

Retrying due to transient client side error HTTPSConnectionPool(host='westus2-0.in.applicationinsights.azure.com', port=443): Max retries exceeded with url: /v2.1/track (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x14307bac0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')).
Retrying due to transient client side error HTTPSConnectionPool(host='westus2-0.in.applicationinsights.azure.com', port=443): Max retries exceeded with url: /v2.1/track (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x143062a60>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')).
Path /var/folders/1r/s3cngdnd4qn_4zjl5q1dk6200000gq/T/opencensus-python-71b954a8-6b7d-43f5-986c-3d3a6605d803/2022-11-18T221618.576910-d4bf776d.blob.tmp does not exist or is inaccessible.


In [99]:
# View Trials inteface
hd_job = ml_client.jobs.get(returned_job.name + "_HD")
hd_job

Experiment,Name,Type,Status,Details Page
Mini-Tumor-Classification-AML_V1,busy_sun_x51p25t921_HD,sweep,Running,Link to Azure Machine Learning studio


# WARNING: STILL DOING MULTILABEL HERE BY MISTAKE

In [ ]:
# Create the AutoML job with the related factory-function.

image_classification_multilabel_job = automl.image_classification_multilabel(
    compute=compute_name,
    # name="dpv2-image-classif-multi-label-job-02",
    experiment_name=exp_name,
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name="label",
    primary_metric=ClassificationMultilabelPrimaryMetrics.IOU,
    tags={"my_custom_tag": "My custom value"},
)

image_classification_multilabel_job.set_limits(
    timeout_minutes=60,
    max_trials=10,
    max_concurrent_trials=2,
)

image_classification_multilabel_job.extend_search_space(
    [
        SearchSpace(
            model_name=Choice(["vitb16r224"]),
            learning_rate=Uniform(0.005, 0.05),
            number_of_epochs=Choice([15, 30]),
            gradient_accumulation_step=Choice([1, 2]),
        ),
        SearchSpace(
            model_name=Choice(["seresnext"]),
            learning_rate=Uniform(0.005, 0.05),
            # model-specific, valid_resize_size should be larger or equal than valid_crop_size
            validation_resize_size=Choice([288, 320, 352]),
            validation_crop_size=Choice([224, 256]),  # model-specific
            training_crop_size=Choice([224, 256]),  # model-specific
        ),
    ]
)

image_classification_multilabel_job.set_sweep(
    sampling_algorithm="Random",
    early_termination=BanditPolicy(
        evaluation_interval=2, slack_factor=0.2, delay_evaluation=6
    ),
)

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    image_classification_multilabel_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

In [ ]:
ml_client.jobs.stream(returned_job.name)

When doing a hyperparameter sweep, it can be useful to visualize the different configurations that were tried using the HyperDrive UI. You can navigate to this UI by going to the 'Child jobs' tab in the UI of the main automl image job from above, which is the HyperDrive parent run. Then you can go into the 'Trials' tab of this HyperDrive parent run. Alternatively, here below you can see directly the HyperDrive parent run and navigate to its 'Trials' tab:

In [ ]:
hd_job = ml_client.jobs.get(returned_job.name + "_HD")
hd_job

# 5. Retrieve the Best Trial (Best Model's trial/run)
Use the MLFLowClient to access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Trial.

## Initialize MLFlow Client

The models and artifacts that are produced by AutoML can be accessed via the MLFlow interface.
Initialize the MLFlow client here, and set the backend as Azure ML, via. the MLFlow Client.

IMPORTANT, you need to have installed the latest MLFlow packages with:

    pip install azureml-mlflow

    pip install mlflow

In [ ]:
!pip install azure-mlflow
!pip install mlflow

In [ ]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

In [ ]:
# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))

In [ ]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

### Get the AutoML parent Job

In [ ]:
job_name = returned_job.name

# Example if providing an specific Job name/ID
# job_name = "salmon_camel_5sdf05xvb3"

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

In [ ]:
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(mlflow_parent_run.data.tags.keys())

### Get the AutoML best child run

In [ ]:
# Get the best model's child run

best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

## Get best model run's metrics
Access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Run.

In [ ]:
import pandas as pd

pd.DataFrame(best_run.data.metrics, index=[0]).T

## Download the best model locally
Access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Run.

In [ ]:
# Create local folder
import os

local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [ ]:
# Download run's artifacts/outputs
local_path = mlflow_client.download_artifacts(
    best_run.info.run_id, "outputs", local_dir
)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

In [ ]:
import os

# Show the contents of the MLFlow model folder
os.listdir("./artifact_downloads/outputs/mlflow-model")

# 6. Register best model and deploy

## 6.1 Create managed online endpoint

In [ ]:
# import required libraries
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
    ProbeSettings,
)

In [ ]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "ic-ml-fridge-items-" + datetime.datetime.now().strftime(
    "%m%d%H%M"
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint for deploying model",
    auth_mode="key",
    tags={"foo": "bar"},
)

In [ ]:
ml_client.begin_create_or_update(endpoint).result()

## 6.2 Register best model and deploy

### Register model

In [ ]:
model_name = "ic-ml-fridge-items-model"
model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/mlflow-model/",
    name=model_name,
    description="my sample image classification multilabel model",
    type=AssetTypes.MLFLOW_MODEL,
)

# for downloaded file
# model = Model(
#     path=path="artifact_downloads/outputs/mlflow-model/",
#     name=model_name,
#     description="my sample image classification multilabel model",
#     type=AssetTypes.MLFLOW_MODEL,
# )

registered_model = ml_client.models.create_or_update(model)

In [ ]:
registered_model.id

### Deploy

In [ ]:
deployment = ManagedOnlineDeployment(
    name="ic-ml-fridge-items-mlflow-dpl",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type="Standard_DS3_V2",
    instance_count=1,
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=2000,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=2000,
    ),
)

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

In [ ]:
# ic ml fridge items deployment to take 100% traffic
endpoint.traffic = {"ic-ml-fridge-items-mlflow-dpl": 100}
ml_client.begin_create_or_update(endpoint).result()

### Get endpoint details

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

In [ ]:
# Create request json
import base64

sample_image = "./data/multilabelFridgeObjects/images/99.jpg"


def read_image(image_path):
    with open(image_path, "rb") as f:
        return f.read()


request_json = {
    "input_data": {
        "columns": ["image"],
        "data": [base64.encodebytes(read_image(sample_image)).decode("utf-8")],
    }
}

In [ ]:
import json

request_file_name = "sample_request_data.json"

with open(request_file_name, "w") as request_file:
    json.dump(request_json, request_file)

In [ ]:
resp = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=deployment.name,
    request_file=request_file_name,
)

## Visualize detections
Now that we have scored a test image, we can visualize the prediction for this image.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import numpy as np
import json

IMAGE_SIZE = (18, 12)
plt.figure(figsize=IMAGE_SIZE)
img_np = mpimg.imread(sample_image)
img = Image.fromarray(img_np.astype("uint8"), "RGB")
x, y = img.size

fig, ax = plt.subplots(1, figsize=(15, 15))
# Display the image
ax.imshow(img_np)
prediction_list = json.loads(resp)
prediction = prediction_list[0]
score_threshold = 0.5

label_offset_x = 30
label_offset_y = 30
for index, score in enumerate(prediction["probs"]):
    if score > score_threshold:
        label = prediction["labels"][index]
        display_text = "{} ({})".format(label, round(score, 3))
        print(display_text)

        color = "red"
        plt.text(label_offset_x, label_offset_y, display_text, color=color, fontsize=30)
        label_offset_y += 30
plt.show()

### Delete the deployment and endopoint

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

# Next Step: Load the best model and try predictions

Loading the models locally assume that you are running the notebook in an environment compatible with the model. The list of dependencies that is expected by the model is specified in the MLFlow model produced by AutoML (in the 'conda.yaml' file within the mlflow-model folder).

Since the AutoML model was trained remotelly in a different environment with different dependencies to your current local conda environment where you are running this notebook, if you want to load the model you have several options:

1. A recommended way to locally load the model in memory and try predictions is to create a new/clean conda environment with the dependencies specified in the conda.yml file within the MLFlow model's folder, then use MLFlow to load the model and call .predict() as explained in the notebook **mlflow-model-local-inference-test.ipynb** in this same folder.

2. You can install all the packages/dependencies specified in conda.yml into your current conda environment you used for using Azure ML SDK and AutoML. MLflow SDK also have a method to install the dependencies in the current environment. However, this option could have risks of package version conflicts depending on what's installed in your current environment.

3. You can also use: mlflow models serve -m 'xxxxxxx'

# Next Steps
You can see further examples of other AutoML tasks such as Regression, Image-Object-Detection, NLP-Text-Classification, Time-Series-Forcasting, etc.